In [1]:
from datetime import datetime
from os.path import exists as file_exists
import pandas as pd
from pprint import pprint as pp
import re
import requests
from time import sleep
import xmltodict
import Credentials      # Get API keys, etc.

apikey = Credentials.prod_api
baseurl = 'https://api-na.hosted.exlibrisgroup.com'
item_query = '/almaws/v1/bibs/{mms_id}/holdings/{holding_id}/items/{item_pid}?apikey={apikey}'

exported_csv = "FullItemList.csv"
filled_csv = "FilledEnumChron.csv"
err_log_txt = "log.txt"

### Run this cell if you want to use the Limited-list CSV
exported_csv = "LimitedList.csv"

#### Create original df

In [2]:
# Depending on how the file was exported, column names may or may have either spaces or underscores
df = pd.read_csv(exported_csv, converters={'Item_ID': str, 'Holdings_ID': str, 'MMS_ID': str, 'Item ID': str, 'Holdings ID': str, 'MMS ID': str})

#### Clean up, tweak & format df

In [3]:
# Remove spaces from column names
df.columns = [c.replace(' ', '_') for c in df.columns]
# Rename certain columns
df = df.rename(columns={'Permanent_Location': 'Location', 'Item_Policy': 'Policy', 'Material_Type': 'Material'})
# Strip leading/trailing space from Description
df.Description = df.Description.str.strip()
# Collapse multiple spaces within the Description
df.Description.replace(' +', ' ', regex=True, inplace=True)

In [4]:
# Add columns for the Enum/Chron fields
EC_fields = ['Enum_A', 'Enum_B', 'Chron_I', 'Chron_J']
df[EC_fields] = None

#### The function for getting info from Description to Enum/Chron fields:

In [5]:
def fill_and_extract(regex, these_fields):
    exp = re.compile(regex)
    for i, f in enumerate(these_fields):
        df[f] = df['Description'].str.extract(exp, expand=True)[i].fillna(df[f])

#### Call it repeatedly for each regex that you come up with:
**[Test your Regex here](https://regex101.com/)**

In [6]:
# Just volume & nothing else
fill_and_extract(r'^v\.(\d)+$', ['Enum_A'])

In [8]:
# Just year or range of years (post-18th-century) & nothing else
fill_and_extract(r'^((?:1[89]|20)\d{2})(?:[\-\/](?:1[89]|20)\d{2})?$', ['Chron_I'])

In [9]:
# Volume + year (or range of years)
fill_and_extract(r'^v\.(\d)+ ((?:1[89]|20)\d{2}(?:[\-\/](?:1[89]|20)?\d{2})?)$', ['Enum_A', 'Chron_I'])

#### Then once all those replacements are done, pull filled-in records out to a new dataframe

In [10]:
# Create a dataframe to hold JUST records that get filled
filled = pd.DataFrame()
# Populate the new dataframe with any records that now have at least one Enum/Chron field filled
filled = df.dropna(subset=EC_fields, thresh=1)

#### Then apply the changes via the API and log filled items to the Filled CSV

In [11]:
records = len(filled)
c = 0
needs_header=not file_exists(filled_csv) # Apparently we're creating the file, so it needs a header

In [ ]:
### Run this bit to see what got "filled" before hitting the API

for index, row in filled.fillna('').iterrows():
    c += 1
    print(c, ' / '.join((row['MMS_ID'], row['Holdings_ID'], row['Item_ID'])), str(row['Description']), sep="\t")


In [ ]:
with open(err_log_txt, 'a') as err_log:
    for index, row in filled.fillna('').iterrows():
        c += 1
        print(c, '/'.join((row['MMS_ID'], row['Holdings_ID'], row['Item_ID'])), str(row['Description']), sep="\t")
        r = requests.get(''.join([baseurl,
                                  item_query.format(mms_id=str(row['MMS_ID']),
                                                    holding_id=str(row['Holdings_ID']),
                                                    item_pid=str(row['Item_ID']),
                                                    apikey=apikey)]))
        rdict = xmltodict.parse(r.text)
        if r.status_code != 200:
            e = xmltodict.parse(r._content)
            # Log the error
            print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"), ' Error FETCHING item ', row['Item_ID'], ': (', r.status_code, ') ',
                e['web_service_result']['errorList']['error']['errorMessage'],
                sep='', file=err_log)
            # Remove this item from the "filled" df
            filled = filled.drop([index])
            continue
        if (c % (records/100) < 1):
            print(int(100*c/records), '% complete', sep='')#, end='\r')
            sleep(5)
            
        # Merge derived values into the retrieved data (rdict)
        rdict['item']['item_data']['enumeration_a'] = str(row['Enum_A'])
        rdict['item']['item_data']['enumeration_b'] = str(row['Enum_B'])
        rdict['item']['item_data']['chronology_i'] = str(row['Chron_I'])
        rdict['item']['item_data']['chronology_j'] = str(row['Chron_J'])
        # Set an internal note, if there's an empty one available
        if ('Enum/Chron derived from Description' not in rdict['item']['item_data'].values()):
            if (not rdict['item']['item_data']['internal_note_1']):
                rdict['item']['item_data']['internal_note_1'] = 'Enum/Chron derived from Description'
            elif (not rdict['item']['item_data']['internal_note_2']):
                rdict['item']['item_data']['internal_note_2'] = 'Enum/Chron derived from Description'
            elif (not rdict['item']['item_data']['internal_note_3']):
                rdict['item']['item_data']['internal_note_3'] = 'Enum/Chron derived from Description'
            else: # Nbd, just log it
                print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"), ' No internal note available for item MMS ID ',
                    str(row['MMS_ID']), sep="", file=err_log)
 
        # Push the altered record back into Alma
        pxml = xmltodict.unparse(rdict)
        p = requests.put(''.join([baseurl,
                                  item_query.format(mms_id=row['MMS_ID'],
                                                    holding_id=row['Holdings_ID'],
                                                    item_pid=row['Item_ID'],
                                                    apikey=apikey)]),
                         data=pxml.encode('utf-8'), headers={'Content-Type': 'application/xml'})
        if p.status_code != 200:
            e = xmltodict.parse(p._content)
            # Log the error
            print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"), ' Error UPDATING item ', row['Item_ID'], ': (', p.status_code, ') ',
                e['web_service_result']['errorList']['error']['errorMessage'],
                sep='', file=err_log)
            # Remove this item from the "filled" df
            filled = filled.drop([index])
            continue

        # Log it to the CSV
        #    Btw, the 'to_frame().T' transposes it, so it all goes in as a single comma-separated row
        row.to_frame().T.to_csv(filled_csv, mode='a', index=False, header=needs_header)
        needs_header=False # Henceforth

#### Purge filled rows from the original CSV

In [53]:
# Purge filled records from the original df
df = df.loc[~df['Item_ID'].isin(filled['Item_ID'])]

# Re-create the original CSV from that df
df.to_csv(exported_csv, index=False) # By default, will overwrite

# View df & filled

In [ ]:
display(df)
display(filled)

# Testing stuff